In [39]:
import torch

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torchsummaryX import summary

In [40]:
class LSTMModel(nn.Module):
    def __init__(self, in_dim, hidden_dim, layers, out_dim, batch_size, seq_len, device):
        super(LSTMModel, self).__init__()
        self.to(device)
        
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.out_dim = out_dim
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.device = device
        
        self.init_states()
        self.lstm = nn.LSTM(in_dim, hidden_dim, layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, out_dim)
        
    def forward(self, inputs):
        hidden = (self.hidden.requires_grad_().detach(), self.cell.requires_grad_().detach())
        output, (self.hidden, self.cell) = self.lstm(inputs, hidden)
        output = self.fc(output)
        return output[:, -1]
    
    def init_states(self):
        self.hidden = torch.zeros(self.layers, self.batch_size, self.hidden_dim).to(self.device)
        self.cell = torch.zeros(self.layers, self.batch_size, self.hidden_dim).to(self.device)

In [46]:
lr = 0.001
n = 312
in_dim = 10
hidden_dim = 25
layers = 3
out_dim = 1
epochs = 100
batch_size = 8
seq_len = 14

device = "cuda" if torch.cuda.is_available() else "cpu"

lstm = LSTMModel(in_dim, hidden_dim, layers, out_dim, batch_size, seq_len, device)
optimizer = Adam(lstm.parameters(), lr=lr)
loss_criterion = nn.MSELoss()

summary(lstm, torch.randn(batch_size, seq_len, in_dim));

       Kernel Shape Output Shape  Params  Mult-Adds
Layer                                              
0_lstm            -  [8, 14, 25]   14100      13500
1_fc        [25, 1]   [8, 14, 1]      26         25
---------------------------------------------------
                      Totals
Total params           14126
Trainable params       14126
Non-trainable params       0
Mult-Adds              13525


In [47]:
xdata1 = np.zeros(shape=(n, in_dim))
ydata1 = np.zeros(shape=(n, out_dim))
#print(xdata1.shape, ydata1.shape)
for i in range(n):
    # Insert data here
    xdata1[i] = np.random.randint(0, 100, size=in_dim)
    ydata1[i] = np.random.randint(0, 100, size=out_dim)

n2 = int(n/seq_len)
xdata2 = np.zeros(shape=(n2, seq_len, in_dim))
ydata2 = np.zeros(shape=(n2, seq_len, out_dim))
#print(xdata2.shape, ydata2.shape)
for i in range(0, n2, seq_len):
    xdata2[i] = xdata1[i:i + seq_len]
    ydata2[i] = ydata1[i:i + seq_len]

n3 = int(n2/batch_size)
xdata = np.zeros(shape=(n3, batch_size, seq_len, in_dim))
ydata = np.zeros(shape=(n3, batch_size, seq_len, out_dim))
#print(xdata.shape, ydata.shape)
for i in range(0, n3, batch_size):
    xdata[i] = xdata2[i:i + batch_size]
    ydata[i] = ydata2[i:i + batch_size]
    
#x = torch.tensor(xdata, requires_grad=True).float()
#y = torch.tensor(ydata, requires_grad=True).float()
#print(x.shape, y.shape)
#split = int(0.5 * n3)
#x0 = x[:split]
#y0 = y[:split]
#x2 = x[split:]
#y2 = y[split:]

In [48]:
x = torch.randn(n, batch_size, seq_len, in_dim)
y = torch.randn(n, batch_size, seq_len, out_dim)
print(x.shape, y.shape)
split = int(0.5 * n)
x0 = x[:split]
y0 = y[:split]
x2 = x[split:]
y2 = y[split:]

torch.Size([312, 8, 14, 10]) torch.Size([312, 8, 14, 1])


In [49]:
for epoch in range(epochs):
    train_sum_loss = 0
    for data, label in zip(x0, y0):
        optimizer.zero_grad()
        output = lstm(data)
        loss = loss_criterion(output, label[:, -1])
        loss.backward(retain_graph=True)
        optimizer.step()
        train_sum_loss += loss.item()
    
    test_sum_loss = 0
    for data, label in zip(x2, y2):
        output = lstm(data)
        loss = loss_criterion(output, label[:, -1])
        test_sum_loss += loss.item()
        
    print("Epoch: {}/{}, Test-Loss: {}, Train-Loss: {}".format(epoch, epochs, test_sum_loss, train_sum_loss))

Epoch: 0/100, Test-Loss: 162.17049928754568, Train-Loss: 154.34604102373123
Epoch: 1/100, Test-Loss: 161.39072385430336, Train-Loss: 154.4379250407219
Epoch: 2/100, Test-Loss: 161.31980746239424, Train-Loss: 154.2151316702366
Epoch: 3/100, Test-Loss: 162.16256911307573, Train-Loss: 153.932579100132
Epoch: 4/100, Test-Loss: 171.93123262375593, Train-Loss: 153.11042115092278
Epoch: 5/100, Test-Loss: 168.83581674098969, Train-Loss: 152.4992038309574
Epoch: 6/100, Test-Loss: 172.69556568562984, Train-Loss: 151.44911602139473
Epoch: 7/100, Test-Loss: 176.2142222672701, Train-Loss: 149.8959027826786
Epoch: 8/100, Test-Loss: 180.89098870754242, Train-Loss: 147.32427328824997
Epoch: 9/100, Test-Loss: 184.18584670126438, Train-Loss: 144.33093231916428
Epoch: 10/100, Test-Loss: 189.22320821881294, Train-Loss: 140.78379474580288
Epoch: 11/100, Test-Loss: 198.01893976330757, Train-Loss: 135.01023618876934
Epoch: 12/100, Test-Loss: 199.78949935734272, Train-Loss: 129.14236782491207
Epoch: 13/100, T

KeyboardInterrupt: 